# Notebook 16 — CNN Retraining on Balanced Dataset
## Trains `fraud_document_cnn.h5` on the new `dataset/` 60/20/20 split

**Target:** ≥85% validation accuracy  
**Input:** `dataset/train/` and `dataset/val/`  
**Output:** `models/fraud_document_cnn.h5` (overwritten with better weights)

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.preprocessing.image import ImageDataGenerator

print(f'TensorFlow version: {tf.__version__}')
print(f'GPU available: {len(tf.config.list_physical_devices("GPU")) > 0}')

BASE_DIR   = Path(r'c:\Users\saigo\Desktop\fraud_document_ai')
TRAIN_DIR  = BASE_DIR / 'dataset' / 'train'
VAL_DIR    = BASE_DIR / 'dataset' / 'val'
TEST_DIR   = BASE_DIR / 'dataset' / 'test'
MODEL_PATH = str(BASE_DIR / 'models' / 'fraud_document_cnn.h5')

IMG_SIZE  = 128
BATCH     = 32
EPOCHS    = 30

# Verify directories
for d in [TRAIN_DIR, VAL_DIR, TEST_DIR]:
    t = len(list((d/'fraud').iterdir()))
    g = len(list((d/'genuine').iterdir()))
    print(f'{d.name}: fraud={t}, genuine={g}, total={t+g}')

In [ ]:
# ── Data generators with augmentation ───────────────────────────────────────

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=5,
    width_shift_range=0.05,
    height_shift_range=0.05,
    zoom_range=0.05,
    horizontal_flip=False,   # bills should not be flipped
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(
    str(TRAIN_DIR),
    target_size=(IMG_SIZE, IMG_SIZE),
    color_mode='grayscale',
    batch_size=BATCH,
    class_mode='binary',
    classes=['fraud', 'genuine'],   # fraud=0, genuine=1
    shuffle=True,
    seed=42
)

val_gen = val_datagen.flow_from_directory(
    str(VAL_DIR),
    target_size=(IMG_SIZE, IMG_SIZE),
    color_mode='grayscale',
    batch_size=BATCH,
    class_mode='binary',
    classes=['fraud', 'genuine'],
    shuffle=False
)

print(f'\nClass indices: {train_gen.class_indices}')
print(f'Train batches: {len(train_gen)}  |  Val batches: {len(val_gen)}')

In [ ]:
# ── CNN Architecture ─────────────────────────────────────────────────────────
# 4-block CNN with batch norm and dropout — designed for CPU training

def build_cnn(img_size=128):
    model = models.Sequential([
        # Block 1
        layers.Conv2D(32, (3,3), activation='relu', padding='same',
                      input_shape=(img_size, img_size, 1)),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2,2),

        # Block 2
        layers.Conv2D(64, (3,3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2,2),
        layers.Dropout(0.25),

        # Block 3
        layers.Conv2D(128, (3,3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2,2),
        layers.Dropout(0.25),

        # Block 4
        layers.Conv2D(64, (3,3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.GlobalAveragePooling2D(),

        # Classifier
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.4),
        layers.Dense(1, activation='sigmoid')   # 0=fraud, 1=genuine
    ])
    return model

model = build_cnn(IMG_SIZE)
model.summary()

In [ ]:
# ── Compile ──────────────────────────────────────────────────────────────────

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss='binary_crossentropy',
    metrics=['accuracy',
             tf.keras.metrics.Precision(name='precision'),
             tf.keras.metrics.Recall(name='recall')]
)

# Callbacks
checkpoint_cb = callbacks.ModelCheckpoint(
    MODEL_PATH,
    monitor='val_accuracy',
    save_best_only=True,
    verbose=1
)
earlystop_cb = callbacks.EarlyStopping(
    monitor='val_accuracy',
    patience=6,
    restore_best_weights=True,
    verbose=1
)
reduce_lr_cb = callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    verbose=1,
    min_lr=1e-6
)

print('✅ Model compiled')

In [ ]:
# ── Compute class weights (handle imbalance) ─────────────────────────────────

n_fraud   = len(list((TRAIN_DIR/'fraud').iterdir()))
n_genuine = len(list((TRAIN_DIR/'genuine').iterdir()))
total     = n_fraud + n_genuine

# fraud=0, genuine=1
class_weights = {
    0: total / (2 * n_fraud),
    1: total / (2 * n_genuine)
}
print(f'Class weights: {class_weights}')
print(f'  (fraud weight {class_weights[0]:.2f} vs genuine {class_weights[1]:.2f})')

In [ ]:
# ── Train ────────────────────────────────────────────────────────────────────
# ⚠️  CPU training: ~3-5 min/epoch × 30 epochs max → ~1.5hrs worst case
# EarlyStopping typically kicks in at epoch 10-15 → ~45min

print('Starting training... (this will take ~30-90 min on CPU)')
print('Best model will be saved to:', MODEL_PATH)

history = model.fit(
    train_gen,
    epochs=EPOCHS,
    validation_data=val_gen,
    class_weight=class_weights,
    callbacks=[checkpoint_cb, earlystop_cb, reduce_lr_cb],
    verbose=1
)

In [ ]:
# ── Plot training curves ─────────────────────────────────────────────────────

fig, axes = plt.subplots(1, 2, figsize=(14, 4))

axes[0].plot(history.history['accuracy'],     label='Train Acc',  color='#58a6ff')
axes[0].plot(history.history['val_accuracy'], label='Val Acc',    color='#3fb950')
axes[0].axhline(0.85, color='#f85149', linestyle='--', label='Target 85%')
axes[0].set_title('Accuracy', fontsize=13)
axes[0].set_xlabel('Epoch')
axes[0].legend()
axes[0].set_ylim([0, 1])

axes[1].plot(history.history['loss'],     label='Train Loss', color='#58a6ff')
axes[1].plot(history.history['val_loss'], label='Val Loss',   color='#3fb950')
axes[1].set_title('Loss', fontsize=13)
axes[1].set_xlabel('Epoch')
axes[1].legend()

plt.tight_layout()

reports_dir = BASE_DIR / 'reports'
reports_dir.mkdir(exist_ok=True)
plt.savefig(str(reports_dir / 'training_curves.png'), dpi=150, bbox_inches='tight')
plt.show()
print('✅ Training curves saved to reports/training_curves.png')

In [ ]:
# ── Evaluate on test set ─────────────────────────────────────────────────────

test_gen = val_datagen.flow_from_directory(
    str(TEST_DIR),
    target_size=(IMG_SIZE, IMG_SIZE),
    color_mode='grayscale',
    batch_size=BATCH,
    class_mode='binary',
    classes=['fraud', 'genuine'],
    shuffle=False
)

results = model.evaluate(test_gen, verbose=1)
print(f'\n=== CNN Test Results ===')
print(f'  Accuracy:  {results[1]:.4f} ({results[1]*100:.1f}%)')
print(f'  Precision: {results[2]:.4f}')
print(f'  Recall:    {results[3]:.4f}')

best_val_acc = max(history.history['val_accuracy'])
print(f'  Best Val Accuracy: {best_val_acc:.4f} ({best_val_acc*100:.1f}%)')
print(f'  Model saved to: {MODEL_PATH}')